# Problem Statement :

# SMS Spam Detection using Natural Language Processing with Python

# Required Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# 1.Data Gathering

In [2]:
df = pd.read_csv(r"C:\Users\aarun\SMS_Spam_Classification_NLP\SMSSpamCollection", sep = '\t', names = ['Label','Msg'] )
df.head()

,Label,Msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# 2. Exploratory Data Analysis

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   Msg     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [38]:
df.isnull().sum()

Label    0
Msg      0
dtype: int64

In [5]:
df['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

# 3. Data Preprocessing

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aarun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aarun\AppData\Roaming\nltk_data...


True

In [10]:
corpus = []
lm = WordNetLemmatizer()
for i in range (len(df)):
    review = re.sub('^a-zA-Z0-9',' ',df['Msg'][i])
    review = review.lower()
    review = review.split()
    review = [data for data in review if data not in stopwords.words('english')]
    review = [lm.lemmatize(data) for data in review]
    review = " ".join(review)
    corpus.append(review)    

In [11]:
df['Msg'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [12]:
len(df['Msg'])

5572

In [13]:
len(corpus)

5572

In [14]:
df['Msg']=corpus
df.head()

,Label,Msg
0,ham,"go jurong point, crazy.. available bugis n gre..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor... u c already say...
4,ham,"nah think go usf, life around though"


# 4. Model Building

## 4.1 Data Splitting

In [15]:
x = df['Msg']
y = df['Label']

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 10)

In [17]:
len(x_train), len(y_train)

(3900, 3900)

In [18]:
len(x_test),len(y_test)

(1672, 1672)

## 4.2 Vectorization (Convert Text Data Into The Vectors)

In [19]:
tf_obj = TfidfVectorizer()
x_train_tfidf = tf_obj.fit_transform(x_train).toarray()
x_train_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
x_train_tfidf.shape

(3900, 6931)

## 4.3 Pipeline

In [21]:
text_mnb = Pipeline([('tfidf',TfidfVectorizer()),('mnb',MultinomialNB())])

In [22]:
text_mnb.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('mnb', MultinomialNB())])

In [23]:
#Accuracy Score on Testing Data
y_pred_test = text_mnb.predict(x_test)
print("Accuracy Score:", accuracy_score(y_test,y_pred_test)*100)

Accuracy Score: 95.8732057416268


In [24]:
#Accuracy Score on Training Data
y_pred_train = text_mnb.predict(x_train)
print("Accuracy Score:",accuracy_score(y_train,y_pred_train)*100)

Accuracy Score: 98.3076923076923


In [25]:
#Confusion Matrix on Testing Data
y_pred_test = text_mnb.predict(x_test)
print("Confusion Matrix on Test Data:\n", confusion_matrix(y_test,y_pred_test))

Confusion Matrix on Test Data:
 [[1457    0]
 [  69  146]]


In [26]:
#Classification Report on Testing Data
y_pred_test = text_mnb.predict(x_test)
print("Classification Reportx on Test Data:\n", classification_report(y_test,y_pred_test))

Classification Reportx on Test Data:
               precision    recall  f1-score   support

         ham       0.95      1.00      0.98      1457
        spam       1.00      0.68      0.81       215

    accuracy                           0.96      1672
   macro avg       0.98      0.84      0.89      1672
weighted avg       0.96      0.96      0.96      1672



# Prediction on User_data 

In [27]:
def preprocess_data(text):
    review = re.sub('^a-zA-Z0-9',' ',text)
    review = review.lower()
    review = review.split()
    review = [data for data in review if data not in stopwords.words('english')]
    review = [lm.lemmatize(data) for data in review]
    review = " ".join(review)
    return [review]

In [28]:
user_data = df['Msg'][0]
print(user_data)
user_data = preprocess_data(user_data)
user_data

go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...


['go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...']

In [29]:
text_mnb.predict(user_data)[0]

'ham'

In [30]:
class prediction:
    
    def __init__(self,data):
        self.data = data
        
    def user_data_preprocessing(self):
        lm = WordNetLemmatizer()
        review = re.sub('^a-zA-Z0-9',' ',self.data)
        review = review.lower()
        review = review.split()
        review = [data for data in review if data not in stopwords.words('english')]
        review = [lm.lemmatize(data) for data in review]
        review = " ".join(review)
        return [review]
    
    def user_data_prediction(self):
        preprocess_data = self.user_data_preprocessing()
        
        if text_mnb.predict(preprocess_data)[0] == 'spam':
            return 'This Message is Spam'
            
        else:
            return 'This Message is Ham'  

In [31]:
df.head()

,Label,Msg
0,ham,"go jurong point, crazy.. available bugis n gre..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor... u c already say...
4,ham,"nah think go usf, life around though"


In [32]:
user_data = df['Msg'][2]
print(user_data)
prediction(user_data).user_data_prediction()

free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's


'This Message is Spam'

In [33]:
user_data = df['Msg'][3]
print(user_data)
prediction(user_data).user_data_prediction()

u dun say early hor... u c already say...


'This Message is Ham'

In [34]:
import pickle

pickle.dump(text_mnb, open('model.pkl', 'wb'))

In [35]:
model1 = pickle.load(open('model.pkl', 'rb'))

In [36]:
te="send $100 to win a car"

In [37]:
prediction(te).user_data_prediction()

'This Message is Ham'